# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import argparse
import itertools
import os
import random
from collections import defaultdict
from copy import deepcopy

import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, trange
import torch.nn.functional as F
from torchmetrics import Accuracy

In [2]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        if access_token is not None:
            self.hangman_url = self.determine_hangman_url()
            self.session = session or requests.Session()
        else:
            print("No access token given.")
        self.access_token = access_token
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):  # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_", ".")

        # find length of passed word
        len_word = len(clean_word)

        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []

        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue

            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word, dict_word):
                new_dictionary.append(dict_word)

        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary

        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)

        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()

        guess_letter = '!'

        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter, instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break

        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter, instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break

        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location, "r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print(
                    "Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id,
                                                                                                                tries_remains,
                                                                                                                word))
            while tries_remains > 0:
                # get guessed letter from user code
                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter",
                                       {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status == "success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status == "failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status == "ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status == "success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result


class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

In [3]:
class CharPredictor(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, pad_id):
        super(CharPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_id)
        bidirectional = True
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, bidirectional=bidirectional,
                            batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.num_labels = vocab_size
        if bidirectional:
            self.fc_out = nn.Linear(hidden_dim*2, self.num_labels)
        else:
            self.fc_out = nn.Linear(hidden_dim, self.num_labels)

    def forward(self, x):
        # Embedding layer
        x = self.embedding(x)

        # LSTM layer
        _, (hidden_state, c_n) = self.lstm(x)
        # Four fully connected layers with ReLU activation
        hidden_state = torch.cat((hidden_state[-1, :, :], hidden_state[-2, :, :]), -1)
        hidden_state = self.dropout(hidden_state)

        out = F.relu(self.fc_out(hidden_state))

        return out


In [16]:
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

def top_k_categorical_accuracy(preds, target, k=5):
    _, top_k_preds = preds.topk(k, dim=-1)
    correct = top_k_preds.eq(target.view(-1, 1).expand_as(top_k_preds))
    top_k_acc_score = correct.float().mean()
    return top_k_acc_score


class CharPredictorDataset(Dataset):
    def __init__(self, input_data, target_data, char_to_idx, pad_id, max_length=30):
        self.input_data = input_data
        self.target_data = target_data
        self.char_to_idx = char_to_idx
        self.max_length = max_length
        self.pad_id = pad_id

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        input_seq = self.input_data[idx]
        target_seq = self.target_data[idx]
        padding_length = self.max_length - len(input_seq)
        attention_mask = [1] * len(input_seq) + [0] * padding_length
        input_seq += [self.pad_id] * padding_length
        # target_seq += [-100] * padding_length
        input_x = torch.tensor(input_seq, dtype=torch.long)
        label = torch.tensor(target_seq, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.bool)
        return input_x, label, attention_mask


class MyHangman(HangmanAPI):
    def __init__(self, access_token=None, session=None, timeout=None, embedding_dim=32, hidden_dim=128, num_layers=5, max_length=30):
        super().__init__(access_token, session, timeout)
        self.special_tokens = ["<pad>", "_"]
        self.vocab = ALPHABET + self.special_tokens
        vocab_size = len(self.vocab)
        self.char_to_idx = {c:idx for idx, c in enumerate(self.vocab)}
        self.idx_to_char = {idx:c for idx, c in enumerate(self.vocab)}
        self.pad_id = self.char_to_idx["<pad>"]
        self.mask_id = self.char_to_idx["_"]
        self.max_length = max_length

        self.model = CharPredictor(vocab_size, embedding_dim, hidden_dim, num_layers=num_layers, pad_id=self.pad_id)

    def encode_word(self, word):
        encoded_word = []
        for cha in word:
            encoded_word.append(self.char_to_idx[cha])
        return encoded_word

    def generate_all_combinations(self, n, limit=40):
        count = 0
        # Start with a list of n zeros
        for zeros in range(n, -1, -1):  # from n to 0
            for positions in itertools.combinations(range(n), n - zeros):
                if count >= limit:
                    return
                combination = [0] * n
                for position in positions:
                    combination[position] = 1
                yield combination
                count += 1

    def generate_all_combinations_reverse(self, n, limit=40):
        count = 0
        # Start with a list of n ones
        for ones in range(n, -1, -1):  # from n to 0
            for positions in itertools.combinations(range(n), n - ones):
                if count >= limit:
                    return
                combination = [1] * n
                for position in positions:
                    combination[position] = 0
                yield combination
                count += 1

    def create_training_data(self):
        input_data, target_data = [], []
        max_len = 0
        for word_idx, word in enumerate(self.full_dictionary):
            # word: apple
            # input: a _ _ l  e
            # output: p
            word = word.strip()
            encoded_word_ids = self.encode_word(word)
            leng = len(set(encoded_word_ids))
            max_len = max(leng, max_len)
            # masking
            word_id2pos = defaultdict(list)
            for idx, word_id in enumerate(encoded_word_ids):
                word_id2pos[word_id].append(idx)

            sorted_word_id2pos = sorted(word_id2pos.items(), key=lambda item: item[0], reverse=True)
            if word_idx < 10:
                print("word:", word)
                print("sorted_word_id2pos:", sorted_word_id2pos)
                print("===============================")

            all_mask_strategies = list(self.generate_all_combinations(len(sorted_word_id2pos)))
            all_mask_strategies_reverse = list(self.generate_all_combinations_reverse(len(sorted_word_id2pos)))
            # Convert lists to tuples
            all_mask_strategies = [tuple(lst) for lst in all_mask_strategies]
            all_mask_strategies_reverse = [tuple(lst) for lst in all_mask_strategies_reverse]

            # Combine and remove duplicates using set
            combined_mask_strategies = list(set(all_mask_strategies + all_mask_strategies_reverse))

            times = len(combined_mask_strategies)
            for i in range(times):
                this_mask_strategy = combined_mask_strategies[i]
                masked_word_ids = deepcopy(encoded_word_ids)
                recorded_targets = []
                for sorted_idx in range(len(sorted_word_id2pos)):
                    word_id, pos_list = sorted_word_id2pos[sorted_idx]
                    if this_mask_strategy[sorted_idx] == 0:
                        for pos in pos_list:
                            masked_word_ids[pos] = self.mask_id
                        recorded_targets.append(word_id)
                if len(recorded_targets) != 0:
                    input_data.append(masked_word_ids.copy())
                    target_data.append(random.sample(recorded_targets, 1)[0])
        print("input_data:", input_data[:100])
        print("target_data:", target_data[:100])
        print("max len:", max_len)
        return input_data, target_data

    def train(self, data_loader, eval_dataloader, loss_fn, optimizer, num_epochs):
        k = 6
        all_eval_loss, eval_acc = [], []

        for epoch in trange(num_epochs):
            all_loss, train_acc = [], []
            self.model.train()
            with tqdm(data_loader, desc=f'Epoch {epoch + 1}/{num_epochs} [Train]') as t:
                for input_batch, target_batch, attention_mask in t:
                    # Convert input and target sequence to tensors
                    input_tensor = input_batch.cuda()
                    target_tensor = target_batch.cuda()
                    # Forward pass
                    output = self.model(input_tensor)
                    out_dim = output.shape[-1]
                    loss = loss_fn(output.view(-1, out_dim), target_tensor.view(-1))
                    preds = torch.argmax(output, dim=-1)
                    acc_score = (preds == target_tensor).float().mean()

                    # Backward pass and optimization
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    all_loss.append(loss.item())
                    train_acc.append(acc_score.item())
                    if len(all_eval_loss) == 0:
                        t.set_postfix(train_loss=sum(all_loss)/len(all_loss), train_acc=sum(train_acc)/len(train_acc),
                                      eval_loss=0.0, eval_acc=0.0)
                    else:
                        t.set_postfix(train_loss=sum(all_loss)/len(all_loss), train_acc=sum(train_acc)/len(train_acc),
                                      eval_loss=sum(all_eval_loss)/len(all_eval_loss),
                                      eval_acc=sum(eval_acc)/len(eval_acc))

            all_eval_loss, eval_acc = [], []
            self.model.eval()
            with torch.no_grad():
                for input_batch, target_batch, attention_mask in eval_dataloader:
                    input_tensor = input_batch.cuda()
                    target_tensor = target_batch.cuda()
                    # Forward pass
                    output = self.model(input_tensor)
                    out_dim = output.shape[-1]
                    loss = loss_fn(output.view(-1, out_dim), target_tensor.view(-1))
                    preds = torch.argmax(output, dim=-1)
                    acc_score = (preds == target_tensor).float().mean()
                    all_eval_loss.append(loss.item())
                    eval_acc.append(acc_score.item())

            print(
                f"Epoch {epoch + 1}/{num_epochs}, Average Train Loss: {sum(all_loss) / len(all_loss)}, Average Train Acc: {sum(train_acc) / len(train_acc)}")
            print(
                f"Epoch {epoch + 1}/{num_epochs}, Average Eval Loss: {sum(all_eval_loss) / len(all_eval_loss)}, Average Eval Acc: {sum(eval_acc) / len(eval_acc)}")

    def select_index(self, probs):
        sorted_indices = torch.argsort(probs, dim=-1, descending=True)
        for idx in sorted_indices:
            guess_letter = self.idx_to_char[int(idx)]
            if guess_letter not in self.guessed_letters and guess_letter not in self.special_tokens:
                return guess_letter

    def guess(self, word):
        clean_word = word[::2]

        input_ids = self.encode_word(clean_word)
        input_ids = input_ids + (self.max_length - len(input_ids)) * [self.pad_id]
        input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).cuda()
        output = self.model(input_tensor)
        output = output.squeeze(0)
        probs = torch.nn.functional.softmax(output, dim=-1)
        guess_letter = self.select_index(probs)
        return guess_letter

In [17]:
# # Training LSTM
embedding_dim = 128
hidden_dim = 64
batch_size = 1000
num_epoch = 1
lr = 1e-2
num_layers = 5
model_path = "model.pth"

hangman = MyHangman(access_token="6a72118068ffb94bfa6ffc3d5a1733", timeout=2000,
                    embedding_dim=embedding_dim, hidden_dim=hidden_dim, num_layers=num_layers)
print("Creating training data...")
input_data, target_data = hangman.create_training_data()
# Create a Dataset
train_x, test_x, train_y, test_y = train_test_split(input_data, target_data, test_size=0.1, random_state=42)
train_dataset = CharPredictorDataset(train_x, train_y, hangman.char_to_idx, hangman.pad_id)
eval_dataset = CharPredictorDataset(test_x, test_y, hangman.char_to_idx, hangman.pad_id)
# Create a DataLoader
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)
print("Len input data:", len(input_data))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(hangman.model.parameters(), lr=lr)

print("Start training...")
hangman.model.cuda()
hangman.train(data_loader, eval_dataloader, loss_fn, optimizer, num_epochs=num_epoch)

print("Saving model...")
torch.save(hangman.model.state_dict(), model_path)

Creating training data...
word: aaa
sorted_word_id2pos: [(0, [0, 1, 2])]
word: aaaaaa
sorted_word_id2pos: [(0, [0, 1, 2, 3, 4, 5])]
word: aaas
sorted_word_id2pos: [(18, [3]), (0, [0, 1, 2])]
word: aachen
sorted_word_id2pos: [(13, [5]), (7, [3]), (4, [4]), (2, [2]), (0, [0, 1])]
word: aaee
sorted_word_id2pos: [(4, [2, 3]), (0, [0, 1])]
word: aag
sorted_word_id2pos: [(6, [2]), (0, [0, 1])]
word: aahed
sorted_word_id2pos: [(7, [2]), (4, [3]), (3, [4]), (0, [0, 1])]
word: aahs
sorted_word_id2pos: [(18, [3]), (7, [2]), (0, [0, 1])]
word: aal
sorted_word_id2pos: [(11, [2]), (0, [0, 1])]
word: aalesund
sorted_word_id2pos: [(20, [5]), (18, [4]), (13, [6]), (11, [2]), (4, [3]), (3, [7]), (0, [0, 1])]
input_data: [[27, 27, 27], [27, 27, 27, 27, 27, 27], [27, 27, 27, 18], [0, 0, 0, 27], [27, 27, 27, 27], [27, 27, 27, 7, 4, 13], [0, 0, 27, 7, 27, 27], [27, 27, 27, 27, 4, 27], [27, 27, 2, 27, 4, 13], [27, 27, 27, 27, 27, 27], [0, 0, 2, 7, 4, 27], [27, 27, 2, 27, 27, 13], [27, 27, 27, 7, 27, 13], [0

100%|██████████| 1/1 [17:22<00:00, 1042.43s/it]

Epoch 1/1, Average Train Loss: 2.500111453896927, Average Train Acc: 0.23199585775884718
Epoch 1/1, Average Eval Loss: 2.374249454862193, Average Eval Acc: 0.2535733858417524
Saving model...


# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [5]:
embedding_dim = 128
hidden_dim = 64
num_layers = 5
model_path = "my_best_model_random.pth"

api = MyHangman(access_token="6a72118068ffb94bfa6ffc3d5a1733", timeout=2000,
                embedding_dim=embedding_dim, hidden_dim=hidden_dim, num_layers=num_layers)

In [10]:
import os
# os.getcwd()
# os.listdir(".")
api.model.cuda()
api.model.load_state_dict(torch.load(model_path))
api.model.eval()

CharPredictor(
  (embedding): Embedding(28, 128, padding_idx=26)
  (lstm): LSTM(128, 64, num_layers=5, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_out): Linear(in_features=128, out_features=28, bias=True)
)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [11]:
from tqdm import trange

num_runs = 100
verbose= False
update_interval = 1
[prev_practice_runs, prev_recorded_runs, prev_recorded_successes, prev_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)

with trange(num_runs) as t:
    for i in t:
        api.start_game(practice=1, verbose=verbose)
        if (i + 1) % update_interval == 0 or i == num_runs - 1:
            total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes = api.my_status()
            practice_success_rate = (total_practice_successes - prev_practice_successes) / (total_practice_runs - prev_practice_runs)
            t.set_postfix(success_rate='{:.3f}'.format(practice_success_rate))

    total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes = api.my_status()
    practice_success_rate = (total_practice_successes - prev_practice_successes) / (total_practice_runs - prev_practice_runs)

    print('Run %d practice games out of an allotted 100,000. Final practice success rate = %.3f' % (
    num_runs, practice_success_rate))

100%|██████████| 100/100 [05:11<00:00,  3.11s/it, success_rate=0.670]


Run 100 practice games out of an allotted 100,000. Final practice success rate = 0.670


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
num_runs=1000
with trange(num_runs) as t:
    for i in t:
        # print('Playing ', i, ' th game')
        # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
        api.start_game(practice=0,verbose=False)

        if (i + 1) % update_interval == 0 or i == num_runs - 1:
            total_practice_runs, total_recorded_runs, total_recorded_successes, total_practice_successes = api.my_status()
            recorded_success_rate = total_recorded_successes / total_recorded_runs
            t.set_postfix(success_rate='{:.3f}'.format(recorded_success_rate))
        # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
        time.sleep(0.5)


 99%|█████████▊| 987/1000 [59:14<00:46,  3.60s/it, success_rate=0.614] 


HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [14]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print("total_recorded_successes:", total_recorded_successes)
print("total_recorded_runs:", total_recorded_runs)
print('overall success rate = %.3f' % success_rate)

total_recorded_successes: 613
total_recorded_runs: 1000
overall success rate = 0.613
